In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import glob
from shapely.geometry import Point
from shapely.geometry import Polygon
import shapely
from IPython.display import clear_output
import geog
from folium.plugins import HeatMap
import folium
from pandas import Series
from matplotlib import pyplot

In [10]:
for month in range(1,13):
    
    month = str(0)+str(month)
    files = glob.glob('../../rawdata/bus/mtabus-data/2018/'+month+'/*')
    
    print(len(files))
    
    break
        
# bus_geodf = gpd.GeoDataFrame()

# nFiles = len(files)

# for file in files:
    
#     print('Files: ' + str(nFiles))

#     ## reading as a dataframe
#     daily_pings = pd.read_csv(file)
#     daily_pings = daily_pings[['timestamp', 'latitude', 'longitude']]
#     daily_pings = daily_pings.dropna()

#     ## transforming in geodataframe
#     bus_geoDF_current = gpd.GeoDataFrame(daily_pings, crs={'init': 'epsg:4326'})
#     bus_geoDF_current['geometry'] = bus_geoDF_current.apply(lambda row: shapely.geometry.Point((row['longitude'], row['latitude'])), axis=1)
#     bus_geoDF_current = bus_geoDF_current.to_crs({'init':'epsg:3857'})

#     bus_geodf = bus_geodf.append(bus_geoDF_current)
    
#     ## logging
#     nFiles = nFiles - 1
#     clear_output()

31


In [18]:
bus_geodf['timestamp'] = pd.to_datetime(bus_geodf['timestamp'])
bus_geodf['timestamp'] = bus_geodf['timestamp'].dt.tz_convert('US/Eastern')
bus_geodf['timestamp'] = bus_geodf['timestamp'].dt.tz_localize(None)

KeyError: 'timestamp'

In [ ]:
 ### Consts
datapath = '../../rawdata/sensors/'
filename = datapath + 'nodes.txt'

# Load sensors
selectedsensors = [ 
                    "sonycnode-b827eb0fedda.sonyc",
                    "sonycnode-b827eb42bd4a.sonyc",
                    "sonycnode-b827eb44506f.sonyc",
                    "sonycnode-b827eb73e772.sonyc",
                    "sonycnode-b827eb84deb5.sonyc",
                    "sonycnode-b827ebb40450.sonyc"]

f = open(filename)
sensors = {}

for line in f:
    s, lat, lon = line.split(' ')
    if s in selectedsensors:

        print('Spatial join for sensor: ', s)

        lat = float(lat)
        lon = float(lon)

        joinedDF = spatialJoin(bus_geodf, lat, lon)
        joinedDF.set_index('timestamp', inplace=True)

        if(joinedDF.shape[0] == 0):
            print('logging: FAILED!!!!')
            print('SENSORID: ', s)
        else:
            print('logging: SUCCESS!!!!')
            joinedDF.to_file('./files/01/' + s + "_bus.shp")

In [14]:
def spatialJoin(geoDataframe, lat, lon):
    d = 400 # meters
    n_points = 20
    angles = np.linspace(0, 360, n_points)
    center = shapely.geometry.Point(lon,lat)
    polygon = Polygon(geog.propagate(center, angles, d))
    
    sinpoly = gpd.GeoDataFrame(crs={'init': 'epsg:4326'})
    sinpoly = sinpoly.append({'geometry': polygon}, ignore_index=True) 
    sinpoly = sinpoly.to_crs({'init':'epsg:3857'})
    
    dataframe = gpd.tools.sjoin(geoDataframe, sinpoly, how='inner', op="within")
    
    return dataframe

In [ ]:
# bus_geodf['timestamp_original'] = pd.to_datetime(bus_geodf['timestamp'])

# bus_geodf['timestamp_us'] = pd.to_datetime(bus_geodf['timestamp'])
# bus_geodf['timestamp_us'] = bus_geodf['timestamp_us'].dt.tz_convert('US/Eastern')

# bus_geodf['timestamp_us_plus'] = pd.to_datetime(bus_geodf['timestamp'])
# bus_geodf['timestamp_us_plus'] = bus_geodf['timestamp_us_plus'].dt.tz_convert('US/Eastern')
# bus_geodf['timestamp_us_plus'] = bus_geodf['timestamp_us_plus'].dt.tz_localize(None)

# ## case 1
# case1 = joined.copy()
# case1.set_index(['timestamp_original'], inplace=True)
# case1.sort_index(ascending=True, inplace=True)

# ## case 2
# case2 = joined.copy()
# case2.set_index(['timestamp_us'], inplace=True)
# case2.sort_index(ascending=True, inplace=True)

# ## case 3
# case3 = joined.copy()
# case3.set_index(['timestamp_us_plus'], inplace=True)
# case3.sort_index(ascending=True, inplace=True)